In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('dataframes/owid-covid-data.csv')
df=df.drop(['total_deaths','new_deaths','iso_code','continent'],axis=1)
df=df[56:] #removing datas where there is no new cases 
df=df.reset_index()
last_date= df['date'].tolist()
x= len(df['date'])
print(df, last_date[x-1])


     index location       date  total_cases  new_cases
0       56    Nepal  3/21/2020            1          0
1       57    Nepal  3/22/2020            1          0
2       58    Nepal  3/23/2020            2          1
3       59    Nepal  3/24/2020            2          0
4       60    Nepal  3/25/2020            3          1
..     ...      ...        ...          ...        ...
335    391    Nepal  2/19/2021       273263         97
336    392    Nepal  2/20/2021       273351         88
337    393    Nepal  2/21/2021       273431         80
338    394    Nepal  2/22/2021       273556        125
339    395    Nepal  2/23/2021       273666        110

[340 rows x 5 columns] 2/23/2021


In [3]:
#  preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
    X, y = [], []
    for i in range(len(timeseries_data)):
        # find the end of this pattern
        end_ix = i + n_features
        # check if we are beyond the sequence
        if end_ix > len(timeseries_data) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [4]:
df1=df['total_cases']
df1

0           1
1           1
2           2
3           2
4           3
        ...  
335    273263
336    273351
337    273431
338    273556
339    273666
Name: total_cases, Length: 340, dtype: int64

In [5]:
timeseries_data = df1
# choose a number of time steps
n_steps = 3
# split into samples
Xtrain, ytrain = prepare_data(timeseries_data, n_steps)
Xtrain.shape, ytrain.shape

((337, 3), (337,))

In [6]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
Xtrain = Xtrain.reshape((Xtrain.shape[0], Xtrain.shape[1], n_features))
Xtrain.shape

(337, 3, 1)

In [7]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [8]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(n_steps,n_features)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 50)             20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(Xtrain,ytrain,epochs=200,verbose=1)

Epoch 1/200
11/11 [==============================] - 11s 19ms/step - loss: 23147535701.3333
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 22160278357.3333
Epoch 3/200
11/11 [==============================] - 0s 6ms/step - loss: 24000984234.6667
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 23645718357.3333
Epoch 5/200
11/11 [==============================] - 0s 5ms/step - loss: 22762398890.6667
Epoch 6/200
11/11 [==============================] - 0s 5ms/step - loss: 23003198805.3333
Epoch 7/200
11/11 [==============================] - 0s 6ms/step - loss: 21492962645.3333
Epoch 8/200
11/11 [==============================] - 0s 5ms/step - loss: 25181654528.0000
Epoch 9/200
11/11 [==============================] - 0s 5ms/step - loss: 22335841621.3333
Epoch 10/200
11/11 [==============================] - 0s 6ms/step - loss: 23771637077.3333
Epoch 11/200
11/11 [==============================] - 0s 6ms/step - loss: 24067415552.0000
Epoch 

In [11]:
import tensorflow as tf

### Lets Do the prediction and check performance metrics
predict=model.predict(Xtrain)

In [12]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(ytrain,predict))

152146.5518729013